In [1]:
!git clone https://github.com/ultralytics/ultralytics

Cloning into 'ultralytics'...


In [1]:
%cd ./ultralytics

c:\Users\Lenovo\Desktop\expand1\ultralytics


C:\Users\Lenovo\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
!git reset --hard 2071776a3672eb835d7c56cfff22114707765ac

HEAD is now at 2071776a `ultralytics 8.1.23` add YOLOv9-C and E models (#8571)


In [4]:
# Download patch
!wget https://gist.githubusercontent.com/Y-T-G/8f4fc0b78a0a559a06fe84ae4f359e6e/raw/17b1407fefeac86d089c4cf14f174c8bb44948af/add_head.patch

--2024-07-18 11:22:04--  https://gist.githubusercontent.com/Y-T-G/8f4fc0b78a0a559a06fe84ae4f359e6e/raw/17b1407fefeac86d089c4cf14f174c8bb44948af/add_head.patch
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10458 (10K) [text/plain]
Saving to: 'add_head.patch'

     0K ..........                                            100% 12,1M=0,001s

2024-07-18 11:22:04 (12,1 MB/s) - 'add_head.patch' saved [10458/10458]



In [5]:
!git apply add_head.patch

In [6]:
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Obtaining file:///C:/Users/Lenovo/Desktop/expand1/ultralytics
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for ultralytics (pyproject.toml): started
  Building editable for ultralytics (pyproject.toml): finished with status 'done'
  Created wheel for ultralytics: filename=ultralytics-8.1.23-0.editable-py3-none-any.whl size=23081 sha256=c87577694197482c13b5c55aa613baf4b8d88cafab336fe4139dac7c71c67edd
  Stored in directory: C:\Users\Lenovo\AppData\Local\


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Roboflow

In [ ]:
!pip install roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="hga92FJ7iHcjjXThQJ5I")
project = rf.workspace("project-n3wam").project("project-ttd5y")
version = project.version(1)
dataset = version.download("yolov8")


In [ ]:
!mkdir -p /datasets/Project-1
!mv Project-1/** /datasets/Project-1
!mv /datasets/Project-1/data.yaml /content/datasets

# Train the model with frozen COCO backbone

In [2]:
# Reset runtime
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

# Wait for restart and then continue

{'status': 'ok', 'restart': True}

: 

In [1]:
%cd ./ultralytics

c:\Users\Lenovo\Desktop\expand1\ultralytics


C:\Users\Lenovo\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from ultralytics import YOLO
import torch
import copy


In [3]:
model = YOLO("yolov8x.pt")  # load a pretrained model (recommended for training)

In [4]:
# Keep a copy of old state dict for sanity check
old_dict = copy.deepcopy(model.state_dict())

In [5]:
# Inspecting Model Architecture
# We should freeze all but the last layer
model.state_dict().keys()

odict_keys(['model.model.0.conv.weight', 'model.model.0.bn.weight', 'model.model.0.bn.bias', 'model.model.0.bn.running_mean', 'model.model.0.bn.running_var', 'model.model.0.bn.num_batches_tracked', 'model.model.1.conv.weight', 'model.model.1.bn.weight', 'model.model.1.bn.bias', 'model.model.1.bn.running_mean', 'model.model.1.bn.running_var', 'model.model.1.bn.num_batches_tracked', 'model.model.2.cv1.conv.weight', 'model.model.2.cv1.bn.weight', 'model.model.2.cv1.bn.bias', 'model.model.2.cv1.bn.running_mean', 'model.model.2.cv1.bn.running_var', 'model.model.2.cv1.bn.num_batches_tracked', 'model.model.2.cv2.conv.weight', 'model.model.2.cv2.bn.weight', 'model.model.2.cv2.bn.bias', 'model.model.2.cv2.bn.running_mean', 'model.model.2.cv2.bn.running_var', 'model.model.2.cv2.bn.num_batches_tracked', 'model.model.2.m.0.cv1.conv.weight', 'model.model.2.m.0.cv1.bn.weight', 'model.model.2.m.0.cv1.bn.bias', 'model.model.2.m.0.cv1.bn.running_mean', 'model.model.2.m.0.cv1.bn.running_var', 'model.mod

In [6]:
# Add a callback to put the frozen layers in eval mode to prevent BN values from changing.
def put_in_eval_mode(trainer, n_layers=22):
  for i, (name, module) in enumerate(trainer.model.named_modules()):
    if name.endswith("bn") and int(name.split('.')[1]) < n_layers:
      module.eval()
      module.track_running_stats = False
      # print(name, " put in eval mode.")

In [7]:
model.add_callback("on_train_epoch_start", put_in_eval_mode)
model.add_callback("on_pretrain_routine_start", put_in_eval_mode)

In [8]:
# Train the model. Freeze the first 22 layers [0-21].
results = model.train(data='C:/Users/Lenovo/Desktop/expand1/datasets/data2.yaml', freeze=22, epochs=5, imgsz=320)

New https://pypi.org/project/ultralytics/8.2.61 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.23 🚀 Python-3.12.3 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050, 3072MiB)
engine\trainer: task=detect, mode=train, model=yolov8x.pt, data=C:/Users/Lenovo/Desktop/expand1/datasets/data2.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=22, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fa

train: Scanning C:\Users\Lenovo\Desktop\expand1\datasets\Project-2\train\labels.cache... 4907 images, 590 backgrounds, 0 corrupt: 100%|██████████| 4907/4907 [00:00<?, ?it/s]
val: Scanning C:\Users\Lenovo\Desktop\expand1\datasets\Project-2\valid\labels.cache... 462 images, 79 backgrounds, 0 corrupt: 100%|██████████| 462/462 [00:00<?, ?it/s]


Plotting labels to runs\detect\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 320 train, 320 val
Using 8 dataloader workers
Logging results to runs\detect\train2
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/5      1.26G      1.077     0.8267       1.36         23        320: 100%|██████████| 307/307 [04:42<00:00,  1.09it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:28<00:00,  1.91s/it]

                   all        462        432       0.95      0.961      0.988      0.783


...

        5/5      1.44G     0.9192      0.582      1.231         17        320: 100%|██████████| 307/307 [04:57<00:00,  1.03it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:23<00:00,  1.57s/it]

                   all        462        432      0.972      0.975      0.993      0.832



5 epochs completed in 0.554 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 136.7MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 136.7MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics YOLOv8.1.23 🚀 Python-3.12.3 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050, 3072MiB)
Model summary (fused): 268 layers, 68124531 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:21<00:00,  1.45s/it]


                   all        462        432      0.972      0.975      0.993      0.832
Speed: 0.1ms preprocess, 40.6ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs\detect\train2


In [9]:
def compare_dicts(state_dict1, state_dict2):
    # Compare the keys
    keys1 = set(state_dict1.keys())
    keys2 = set(state_dict2.keys())

    if keys1 != keys2:
        print("Models have different parameter names.")
        return False

    # Compare the values (weights)
    for key in keys1:
        if not torch.equal(state_dict1[key], state_dict2[key]):
            print(f"Weights for parameter '{key}' are different.")
            if "bn" in key and "22" not in key:
              state_dict1[key] = state_dict2[key]

In [10]:
# Compare the dicts. Changes should only be in layer 21 and above
compare_dicts(old_dict, model.state_dict())

Weights for parameter 'model.model.22.cv2.0.0.bn.weight' are different.
Weights for parameter 'model.model.22.cv2.1.0.bn.running_mean' are different.
Weights for parameter 'model.model.22.cv2.0.0.bn.running_var' are different.
Weights for parameter 'model.model.22.cv3.1.2.weight' are different.
Weights for parameter 'model.model.22.cv3.2.2.bias' are different.
Weights for parameter 'model.model.22.cv2.1.2.weight' are different.
Weights for parameter 'model.model.22.cv3.0.0.bn.weight' are different.
Weights for parameter 'model.model.22.cv2.2.0.bn.weight' are different.
Weights for parameter 'model.model.22.cv2.2.1.conv.weight' are different.
Weights for parameter 'model.model.22.cv3.1.1.bn.bias' are different.
Weights for parameter 'model.model.22.cv3.0.2.bias' are different.
Weights for parameter 'model.model.22.cv3.0.1.bn.bias' are different.
Weights for parameter 'model.model.22.cv2.1.0.bn.weight' are different.
Weights for parameter 'model.model.22.cv2.2.1.bn.running_mean' are diff

In [11]:
new_state_dict = dict()

In [12]:
#  Increment the head number by 1 in the state_dict
for k, v in model.state_dict().items():
  if k.startswith("model.model.22"):
    new_state_dict[k.replace("model.22", "model.23")] = v
  # else:
  #   new_state_dict[k] = v

In [13]:
# Save the current state_dict
torch.save(new_state_dict, "yolov8x_lp.pth")

# Create new architecture with extra head and load custom weights

In [14]:
from ultralytics import YOLO
import torch

In [15]:
model_2 = YOLO('ultralytics/cfg/models/v8/yolov8x-2xhead.yaml', task="detect").load('yolov8x.pt')

Transferred 595/680 items from pretrained weights


In [16]:
model_2.state_dict().keys()

odict_keys(['model.model.0.conv.weight', 'model.model.0.bn.weight', 'model.model.0.bn.bias', 'model.model.0.bn.running_mean', 'model.model.0.bn.running_var', 'model.model.0.bn.num_batches_tracked', 'model.model.1.conv.weight', 'model.model.1.bn.weight', 'model.model.1.bn.bias', 'model.model.1.bn.running_mean', 'model.model.1.bn.running_var', 'model.model.1.bn.num_batches_tracked', 'model.model.2.cv1.conv.weight', 'model.model.2.cv1.bn.weight', 'model.model.2.cv1.bn.bias', 'model.model.2.cv1.bn.running_mean', 'model.model.2.cv1.bn.running_var', 'model.model.2.cv1.bn.num_batches_tracked', 'model.model.2.cv2.conv.weight', 'model.model.2.cv2.bn.weight', 'model.model.2.cv2.bn.bias', 'model.model.2.cv2.bn.running_mean', 'model.model.2.cv2.bn.running_var', 'model.model.2.cv2.bn.num_batches_tracked', 'model.model.2.m.0.cv1.conv.weight', 'model.model.2.m.0.cv1.bn.weight', 'model.model.2.m.0.cv1.bn.bias', 'model.model.2.m.0.cv1.bn.running_mean', 'model.model.2.m.0.cv1.bn.running_var', 'model.mod

In [17]:
state_dict = torch.load("yolov8x_lp.pth")

In [18]:
state_dict.keys()

dict_keys(['model.model.23.cv2.0.0.conv.weight', 'model.model.23.cv2.0.0.bn.weight', 'model.model.23.cv2.0.0.bn.bias', 'model.model.23.cv2.0.0.bn.running_mean', 'model.model.23.cv2.0.0.bn.running_var', 'model.model.23.cv2.0.0.bn.num_batches_tracked', 'model.model.23.cv2.0.1.conv.weight', 'model.model.23.cv2.0.1.bn.weight', 'model.model.23.cv2.0.1.bn.bias', 'model.model.23.cv2.0.1.bn.running_mean', 'model.model.23.cv2.0.1.bn.running_var', 'model.model.23.cv2.0.1.bn.num_batches_tracked', 'model.model.23.cv2.0.2.weight', 'model.model.23.cv2.0.2.bias', 'model.model.23.cv2.1.0.conv.weight', 'model.model.23.cv2.1.0.bn.weight', 'model.model.23.cv2.1.0.bn.bias', 'model.model.23.cv2.1.0.bn.running_mean', 'model.model.23.cv2.1.0.bn.running_var', 'model.model.23.cv2.1.0.bn.num_batches_tracked', 'model.model.23.cv2.1.1.conv.weight', 'model.model.23.cv2.1.1.bn.weight', 'model.model.23.cv2.1.1.bn.bias', 'model.model.23.cv2.1.1.bn.running_mean', 'model.model.23.cv2.1.1.bn.running_var', 'model.model.2

In [19]:
# Load state_dict of the custom trained model head. Layer 23.
model_2.load_state_dict(state_dict, strict=False)

_IncompatibleKeys(missing_keys=['model.model.0.conv.weight', 'model.model.0.bn.weight', 'model.model.0.bn.bias', 'model.model.0.bn.running_mean', 'model.model.0.bn.running_var', 'model.model.1.conv.weight', 'model.model.1.bn.weight', 'model.model.1.bn.bias', 'model.model.1.bn.running_mean', 'model.model.1.bn.running_var', 'model.model.2.cv1.conv.weight', 'model.model.2.cv1.bn.weight', 'model.model.2.cv1.bn.bias', 'model.model.2.cv1.bn.running_mean', 'model.model.2.cv1.bn.running_var', 'model.model.2.cv2.conv.weight', 'model.model.2.cv2.bn.weight', 'model.model.2.cv2.bn.bias', 'model.model.2.cv2.bn.running_mean', 'model.model.2.cv2.bn.running_var', 'model.model.2.m.0.cv1.conv.weight', 'model.model.2.m.0.cv1.bn.weight', 'model.model.2.m.0.cv1.bn.bias', 'model.model.2.m.0.cv1.bn.running_mean', 'model.model.2.m.0.cv1.bn.running_var', 'model.model.2.m.0.cv2.conv.weight', 'model.model.2.m.0.cv2.bn.weight', 'model.model.2.m.0.cv2.bn.bias', 'model.model.2.m.0.cv2.bn.running_mean', 'model.model

In [20]:
# Output of COCO model
model_coco = YOLO("yolov8x.pt")

# For consistency
model_coco.model.names = {k:k for k in model_coco.names.keys()}

result_coco = model_coco.predict("C:/Users/Lenovo/Desktop/expand1/test.jpg")[0]


image 1/1 C:\Users\Lenovo\Desktop\expand1\test.jpg: 384x640 9 0s, 1 2, 1 5, 2 58s, 144.6ms
Speed: 3.0ms preprocess, 144.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


In [21]:
# Output of custom LP model
model_lp = YOLO("C:/Users/Lenovo/Desktop/expand1/ultralytics/runs/detect/train2/weights/best.pt")

# For plotting consistency, change all class names to class numbers
model_lp.model.names = {k:k for k in model_lp.names.keys()}

result_lp = model_lp.predict("C:/Users/Lenovo/Desktop/expand1/test.jpg")[0]


image 1/1 C:\Users\Lenovo\Desktop\expand1\test.jpg: 192x320 1 0, 72.3ms
Speed: 1.0ms preprocess, 72.3ms inference, 3.0ms postprocess per image at shape (1, 3, 192, 320)


In [22]:
# import the necessary packages
from collections import namedtuple
import numpy as np
import cv2
# define the `Detection` object
Detection = namedtuple("Detection", ["image_path", "gt", "pred"])

In [23]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [24]:
print(model_2.names)

{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '10', 11: '11', 12: '12', 13: '13', 14: '14', 15: '15', 16: '16', 17: '17', 18: '18', 19: '19', 20: '20', 21: '21', 22: '22', 23: '23', 24: '24', 25: '25', 26: '26', 27: '27', 28: '28', 29: '29', 30: '30', 31: '31', 32: '32', 33: '33', 34: '34', 35: '35', 36: '36', 37: '37', 38: '38', 39: '39', 40: '40', 41: '41', 42: '42', 43: '43', 44: '44', 45: '45', 46: '46', 47: '47', 48: '48', 49: '49', 50: '50', 51: '51', 52: '52', 53: '53', 54: '54', 55: '55', 56: '56', 57: '57', 58: '58', 59: '59', 60: '60', 61: '61', 62: '62', 63: '63', 64: '64', 65: '65', 66: '66', 67: '67', 68: '68', 69: '69', 70: '70', 71: '71', 72: '72', 73: '73', 74: '74', 75: '75', 76: '76', 77: '77', 78: '78', 79: '79', 80: '80'}


In [57]:
def remove_bus_with_high_iou(result, iou_threshold=0):
    """
    Removes bus labels if they have high IoU with a shuttle label.

    Args:
        result: A YOLOv8 Results object containing predicted bounding boxes.
        iou_threshold (float): The IoU threshold above which a bus label will be removed.
    """
    boxes = result.boxes.xyxy.cpu().numpy()  
    classes = result.boxes.cls.cpu().numpy()
    print(classes)
    # Replace 'bus' and 'autonomous-shuttle' with your actual class names
    shuttle_indices = np.where(classes == 80.0)[0] 
    print(model_2.names[80])
    print(shuttle_indices)
    bus_indices = np.where(classes == 5.0)[0]
    print(bus_indices)
    boxes_to_remove = []
    for bus_idx in bus_indices:
        for shuttle_idx in shuttle_indices:
            iou = bb_intersection_over_union(boxes[bus_idx], boxes[shuttle_idx])
            if iou > iou_threshold:
                boxes_to_remove.append(bus_idx)
                break

    # Update the boxes directly (not boxes.boxes)
    result.boxes = result.boxes[
        [i for i in range(len(result.boxes)) if i not in boxes_to_remove]
    ]

    return result


In [55]:
# Output of the new model
result_merged = model_2.predict("C:/Users/Lenovo/Desktop/expand1/test.jpg")[0]


image 1/1 C:\Users\Lenovo\Desktop\expand1\test.jpg: 384x640 9 0s, 1 2, 1 5, 2 58s, 1 80, 278.1ms
Speed: 7.0ms preprocess, 278.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


In [58]:
# Apply the custom function
result_merged = remove_bus_with_high_iou(result_merged)

[          2           0          80           0           0           0           0           0           0           0          58          58           0]
80
[2]
[]


In [59]:
import matplotlib.pyplot as plt
import numpy as np

In [60]:
def show_output(images):
    image = np.vstack(images)
    fig = plt.figure()
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    fig.set_size_inches((5,15))
    ax.imshow(image[...,::-1])

In [62]:
plt.savefig("C:/Users/Lenovo/Desktop/expand1/a.jpg") 

In [61]:
show_output([result_coco.plot(), result_lp.plot(), result_merged.plot()])

In [ ]:
# Save .pt of merged model
model_2.ckpt = {"model": model_2.model}
model_2.save("merged.pt")

# Load saved merged model. No additional steps required.
model_2 = YOLO("merged.pt")